In [ ]:
%cd ..
%cd ..
!pip install -r requirements.txt
!pip show pydantic

In [ ]:
"""
app/services/game.py
GameService - 낮 파이프라인 통합

DB(Games 모델) → WorldStatePipeline 변환 → 파이프라인 실행 → DB 저장
파이프라인: LockManager → DayController → EndingChecker → NarrativeLayer
"""
# from __future__ import annotations
import logging

logging.basicConfig(level=logging.DEBUG, force=True)

import copy
from typing import Any, Dict
# from app.crud import game as crud_game
# from app.redis_client import get_redis_client

from app.schemas.client_sync import GameClientSyncSchema
from app.schemas.world_meta_data import WorldDataSchema, LocksSchemaList
from app.schemas.npc_info import NpcCollectionSchema
from app.schemas.player_info import PlayerSchema
from app.schemas.player_info import PlayerSchema
from app.day_controller import get_day_controller
from app.loader import ScenarioLoader, ScenarioAssets
from pathlib import Path

from sqlalchemy.orm import Session
from sqlalchemy.orm.attributes import flag_modified

from app.db_models.game import Games
from app.db_models.scenario import Scenario
# from app.crud import game as crud_game
from app.loader import ScenarioAssets
from app.schemas.game_state import NPCState, WorldStatePipeline, StateDelta
from app.schemas.tool import ToolResult
from app.lock_manager import get_lock_manager
from app.ending_checker import check_ending
from app.narrative import get_narrative_layer
from app.day_controller import get_day_controller
from app.night_controller import get_night_controller
import logging

logger=logging.getLogger(__name__)

# ============================================================
# Delta 적용 로직
# ============================================================

from app.services.game import (
    _apply_delta,
    mock_load_scenario_assets_from_yaml,
    mock_create_world_state_from_yaml,
)
"""
process_turn() 하나씩 뜯어보기
목업 Assets / WorldStatePipeline 생성
"""

# STEP 1 : Assets 불러오기 (목업)
assets = mock_load_scenario_assets_from_yaml()

def print_pydantic(data):
    """Pydantic BaseModel, dataclass, dict 등 어떤 타입이든 JSON으로 출력"""
    import json
    from dataclasses import asdict, is_dataclass
    from enum import Enum

    # 1) Pydantic BaseModel → mode="json" (set→list, Enum→value 자동 변환)
    if hasattr(data, "model_dump"):
        dumped = data.model_dump(mode="json")
    # 2) dataclass
    elif is_dataclass(data) and not isinstance(data, type):
        dumped = asdict(data)
    # 3) dict / list 등
    elif isinstance(data, (dict, list)):
        dumped = data
    else:
        dumped = str(data)
        print(dumped)
        return

    # fallback encoder: Enum, set 등 json.dumps가 못 처리하는 타입 대비
    def _default(obj):
        if isinstance(obj, set):
            return sorted(obj)
        if isinstance(obj, Enum):
            return obj.value
        if hasattr(obj, "model_dump"):
            return obj.model_dump(mode="json")
        if is_dataclass(obj) and not isinstance(obj, type):
            return asdict(obj)
        return str(obj)

    print(json.dumps(dumped, ensure_ascii=False, indent=2, default=_default))

print_pydantic(assets)

In [ ]:
# STEP 3 : 파이프라인 진행

from app.services.scenario import ScenarioService
from app.db_models.game import Games
from app.db_models.scenario import Scenario
from app.schemas.status import GameStatus
from app.services.game import GameService
from app.schemas.request_response import StepRequestSchema
import logging

logging.basicConfig(level=logging.INFO, force=True)

# MOCK WorldState
world_state = mock_create_world_state_from_yaml()

# MOCK Scenario
scenario = Scenario(id=1, title=assets.scenario_id, world_asset_data=assets.model_dump())

# MOCK Game
game = Games(
    scenarios_id=1,
    user_id=1,
    world_meta_data=ScenarioService.extract_initial_world_data(world_state.model_dump()),
    player_data=ScenarioService.extract_initial_player_data(world_state.model_dump()),
    npc_data=ScenarioService.extract_initial_npc_data(world_state.model_dump()),
    status=GameStatus.LIVE,
)
game.scenario = scenario

def run_day_turn(turn_no: int):
    user_input = input(f"[DAY Turn {turn_no}/10] > ")
    req = StepRequestSchema(chat_input=user_input)
    resp, game, world_after = GameService.process_turn(db=None, game_id=1, input_data=req, game=game)
    print(resp.narrative)
    print(f"[game] {game}")
    print(f"[world_after] {world_after}")
    if resp.ending_info:
        print(f"[ENDING] {resp.ending_info.get('ending_id')}")
    return resp


def run_night():
    resp = GameService.process_night(db=None, game_id=1, game=game)
    print(resp.narrative)
    if resp.ending_info:
        print(f"[ENDING] {resp.ending_info.get('ending_id')}")
    return resp


ended = False
cycle = 0

import time

for i in range(1, 11):
    start_time = time.time()

    day_resp = run_day_turn(i)

    end_time = time.time()
    print(f"[DAY {i}] elapsed: {end_time - start_time:.4f} sec")

    if day_resp.ending_info:
        print(f"[ENDING] {day_resp.ending_info.get('ending_id')}")
        print(f"[day_resp] {day_resp}")
        ended = True
        break

if ended:
    pass
else:
    # 밤 1회 진행
    start_time = time.time()

    night_resp = run_night()

    end_time = time.time()
    print(f"[NIGHT] elapsed: {end_time - start_time:.4f} sec")

    if night_resp.ending_info:
        ended = True

